In [5]:
from math import cos, tan, sin
import time
import naoqi
import almath
from numpy import matrix, zeros, invert
from naoqi import ALProxy

IP = "192.168.10.2"
PORT = 9559

proxy = ALProxy("ALMotion", IP, PORT)

MaxSpeed = 0.2

In [6]:
names = "RArm"
stiffness = 1.0

proxy.stiffnessInterpolation(names, stiffness, 1.0)

In [10]:
def stiff(joint):
    proxy.stiffnessInterpolation(joint, 1.0, 1.0)

In [11]:
def unstiff(joint):
    proxy.stiffnessInterpolation(joint, 0.0, 1.0)

In [12]:
def grab(hand): 
    proxy.setAngles(hand, 0, MaxSpeed)
def release(hand):
    proxy.setAngles(hand, 1, MaxSpeed)

In [235]:
sensor_angles = proxy.getAngles(["RArm"], True)

In [236]:
sensor_angles

[0.03225588798522949,
 0.015298128128051758,
 0.004559993743896484,
 0.03490658476948738,
 -0.03072190284729004,
 0.9567999839782715]

In [225]:
proxy.setAngles(["RArm"], sensor_angles, MaxSpeed)

In [109]:
grab("RHand")

In [110]:
release("RHand")

In [247]:
def set_angles(angles): # dict with joint name as key and angle as value
    for k, v in angles.items():
        proxy.setAngles(k, v, MaxSpeed)

def angles(points, dry=False):
    for i in range(0, len(points)):
        px = points[i][0]
        py = points[i][1]
        pz = points[i][2]
        
        if px == 0:
            theta1 = 0
        else:
            theta1 = math.atan(py/px)
            
        if pz == 0:
            theta2 = 0
        else:
            theta2 = math.atan((math.cos(theta1) * px + math.sin(theta1)*py)/pz)
            
        if (math.sin(theta1) * px-math.cos(theta1)*py) == 0:
            theta3 = 0
        else:
            theta3 = math.atan((math.cos(theta1)*math.cos(theta2)*px + 
                                math.sin(theta1)*math.cos(theta2)*py - 
                                                 math.sin(theta2)*pz)
                               /(math.sin(theta1)*px-math.cos(theta1)*py))
            
        
        if pow(math.cos(theta3), 2) - pow(math.sin(theta3), 2) == 0:
            theta4 = 0
        else:
            theta4 = 1/(pow(math.cos(theta3), 2) - pow(math.sin(theta3), 2))
        
        jointAngles = {"RShoulderPitch": theta1, 
                       "RShoulderRoll": theta2,
                       "RElbowYaw": theta3,
                       "RElbowRoll": -theta4
                      }
        
        theta1_valid = theta1 >= -2.0857 and theta1 <= 2.0857
        theta2_valid = theta2 >= -1.3265 and theta2 <= 0.3142
        theta3_valid = theta3 >= -2.0857 and theta3 <= 2.0857
        theta4_valid = -theta4 >=  0.0349 and -theta4 <= 1.5446
        
        valid = theta1_valid and theta2_valid and theta3_valid and theta4_valid
            
        if not dry and valid:
            set_angles(jointAngles)
        else:
            print jointAngles
            print "Theta 1 (RShoulderPitch) valid: " + str(theta1_valid)
            print "Theta 2 (RShoulderRoll) valid: " + str(theta2_valid)
            print "Theta 3 (RElbowYaw) valid: " + str(theta3_valid)
            print "Theta 4 (RElbowRoll) valid: " + str(theta4_valid)
            
        if i < len(points)-1:
            time.sleep(0.5)

In [218]:
angles([[1, -1, -1]])#, [1, -0.5, -1], [1, 0.01, -0.8], [1, 0.5, -1.2], [1, 0.7, -1.57]])

{'RElbowRoll': -1.0, 'RElbowYaw': 0.0, 'RShoulderRoll': -0.9553166181245092, 'RShoulderPitch': -0.7853981633974483}
Theta 1 (RShoulderPitch) valid: True
Theta 2 (RShoulderRoll) valid: True
Theta 3 (RElbowYaw) valid: True
Theta 4 (RElbowRoll) valid: False


In [13]:
unstiff("RArm")

In [233]:
stiff("RArm")

In [216]:
angles([[1, 0.7, -1.57]])

In [58]:
memory = ALProxy("ALMemory", IP, PORT)

In [60]:
memory.getData("redBallDetected")

[[1253146743, 948887],
 [0.08968264609575272,
  0.04500041529536247,
  0.1959732174873352,
  0.2042326182126999],
 [0.07029280811548233,
  -0.0014049189630895853,
  0.17703573405742645,
  0.0,
  0.22338950634002686,
  -0.019984006881713867],
 [-0.010429084300994873,
  -0.015301856212317944,
  0.412810742855072,
  0.03843214735388756,
  -0.00013380493328440934,
  -0.021550489589571953],
 0]

In [61]:
ballInfo = memory.getData("redBallDetected")

In [536]:
ballInfo

[[1253155894, 171886],
 [-0.3952679932117462,
  -0.2094249725341797,
  0.14116714894771576,
  0.14711670577526093],
 [0.07541408389806747,
  -0.004984874743968248,
  0.16874665021896362,
  -1.862645149230957e-09,
  0.3369056284427643,
  -0.06600403785705566],
 [-0.000363372266292572,
  -0.019275322556495667,
  0.4065743088722229,
  0.04832425341010094,
  0.017599664628505707,
  -0.08746590465307236],
 0]

In [537]:
from math import atan 

def distance(ball_size, angle):
    return ball_size / (tan(angle))

In [538]:
real_distance = (distance(0.03, ballInfo[1][2]) + distance(0.03, ballInfo[1][3])) / 2

In [539]:
real_distance

0.2067734622903446

In [540]:
x = real_distance * math.sin(ballInfo[1][0]) * math.cos((math.pi / 2) + ballInfo[1][1])
y = real_distance * math.sin(ballInfo[1][0]) * math.sin((math.pi / 2) + ballInfo[1][1])
z = real_distance * math.cos(ballInfo[1][0])

In [541]:
x

-0.016552643919127925

In [542]:
y

-0.07787963415147181

In [543]:
z

0.1908298647257709

In [429]:
camera = almath.Position6D(ballInfo[2])

In [546]:
ball_camera = almath.Position6D(z, -x, -y, 0, 0, 0) + camera

In [557]:
ball_camera

Position6D(x=0.266179, y=0.011572, z=0.246742, wx=1.86265e-009, wy=0.335372, wz=-0.066004)

In [550]:
ball_camera_shoulder = ball_camera - almath.Position6D(0, -0.98, 1.00, 0, 0, 0)

In [551]:
ball_camera_shoulder

Position6D(x=0.266179, y=0.991572, z=-0.753258, wx=1.86265e-009, wy=0.335372, wz=-0.066004)

In [101]:
stiff("RArm")

In [560]:
angles([[ball_camera.x, ball_camera.y, ball_camera.z]], True)

{'RElbowRoll': -1.0002560735223984, 'RElbowYaw': -1.5594822008978377, 'RShoulderRoll': 0.8237454595008504, 'RShoulderPitch': 0.04344729609189763}


In [9]:
unstiff("RArm")

NameError: name 'unstiff' is not defined

Arm start:

In [81]:
angles_start = [0.21940398216247559, # RShoulderPitch
                0.05824995040893555, # RShoulderRoll
                0.2822141647338867,  # RElbowYaw
                0.06600403785705566, # RElbowRoll
                -0.2516179084777832, # RWristYaw
                0.9567999839782715]  # RHand

[-0.03063797950744629,
 0.08893013000488281,
 0.7669579982757568,
 1.0769100189208984,
 -0.04606199264526367,
 0.9567999839782715]

In [85]:
# links     x       y       z
links = [[  0.00, -98.00, 100.00], # Torso to RShoulderPitch
         [  0.00,   0.00,   0.00], # RShoulderPitch to RShoulderRoll
         [195.00,  15.00,   0.00], # RShoulderRoll to RElbowYaw
         [  0.00,   0.00,   0.00], # RElbowYaw to RElbowRoll
         [ 55.95,   0.00,   0.00]] # RElbowRoll to RWristYaw

In [95]:
proxy.setAngles("RWristYaw", 0, MaxSpeed)

In [234]:
set_angles({"RShoulderPitch": 0, 
            "RShoulderRoll": 0,
            "RElbowYaw": 0,
            "RElbowRoll": 0,
            "RWristYaw": 0,
           })

## Forward kinematics

Transformation to hand from origin is defined as:
$$
T_o^h = T_o^S * A_1 * A_2 * A_3 * A_4 * A_5 * T_w^h 
$$

Transformation to shoulder pitch from origin is defined as:
$$
T_o^s = \left[ \begin{array}{cccc}
                1 & 0 & 0 &   0 \\
                0 & 1 & 0 & -98 \\
                0 & 0 & 1 & 100 \\
                0 & 0 & 0 &   1
                \end{array} \right]
$$

Or in Python as:

In [24]:
T_s_o = matrix([[1, 0, 0,  0    ], 
                [0, 1, 0, -0.098], 
                [0, 0, 1,  0.100], 
                [0, 0, 0,  1    ]])

Transformation based on shoulder pitch is defined as:
$$
A_1 = \left[ \begin{array}{cccc}
                cos_{\theta_1} &  0 & -sin_{\theta_1} & 0 \\
                sin_{\theta_1} &  0 &  cos_{\theta_1} & 0 \\
                            0 & -1 &              0 & 0 \\
                            0 &  0 &              0 & 1
                \end{array} \right]
$$

Or in Python as:

In [25]:
def A1(theta1):
    return matrix([
        [cos(theta1),  0, -sin(theta1), 0], 
        [sin(theta1),  0,  cos(theta1), 0],
        [          0, -1,            0, 0],
        [          0,  0,            0, 1]
    ])

Transformation based on shoulder roll is defined as:
$$
A_2 = \left[ \begin{array}{cccc}
                cos_{\theta_2} & 0 &  sin_{\theta_2} & 0 \\
                sin_{\theta_2} & 0 & -cos_{\theta_2} & 0 \\
                            0 & 1 &                0 & 0 \\
                            0 & 0 &                0 & 1
                \end{array} \right]
$$

Or in Python as:

In [26]:
def A2(theta2):
    return matrix([
        [cos(theta2), 0,  sin(theta2), 0],
        [sin(theta2), 0, -cos(theta2), 0],
        [          0, 1,            0, 0],
        [          0, 0,            0, 1]
    ])

Transformation based on elbow roll, including upper arm length of 105, is defined as:
$$
A_3 = \left[ \begin{array}{cccc}
                cos_{\theta_3+\theta_3} &  0 &  sin_{\theta_3+\theta_3} &   0 \\
                sin_{\theta_3+\theta_3} &  0 & -cos_{\theta_3+\theta_3} &   0 \\
                                      0 & -1 &              0           & 105 \\
                                      0 &  0 &              0           &   1
                \end{array} \right]
$$

Or in Python as:

In [27]:
def A3(theta3):
    return matrix([
        [cos(theta3+theta3),  0,  sin(theta3+theta3), 0    ],
        [sin(theta3+theta3),  0, -cos(theta3+theta3), 0    ],
        [                 0, -1,                   0, 0.105],
        [                 0,  0,                   0, 1    ]
    ])

Transformation based on elbow yaw is defined as:
$$
A_4 = \left[ \begin{array}{cccc}
                cos_{\theta_4} &  0 & -sin_{\theta_4} & 0 \\
                sin_{\theta_4} &  0 &  cos_{\theta_4} & 0 \\
                             0 & -1 &               0 & 0 \\
                             0 &  0 &               0 & 1
                \end{array} \right]
$$

Or in Python as:

In [28]:
def A4(theta4):
    return matrix([
        [cos(theta4),  0, -sin(theta4), 0],
        [sin(theta4),  0,  cos(theta4), 0],
        [          0, -1,            0, 0],
        [          0,  0,            0, 1]
    ])

Transformation based on wrist yaw, including lower arm length of 55.95, is defined as:
$$
A_5 = \left[ \begin{array}{cccc}
                cos_{\theta_5} &  0 & -sin_{\theta_5} &  0    \\
                sin_{\theta_5} &  0 &  cos_{\theta_5} &  0    \\
                             0 & -1 &               0 & 55.95 \\
                             0 &  0 &               0 &  1
                \end{array} \right]
$$

Or in Python as:

In [29]:
def A5(theta5):
    return matrix([
        [cos(theta5),  0, -sin(theta5), 0      ],
        [sin(theta5),  0,  cos(theta5), 0      ],
        [          0, -1,            0, 0.05595],
        [          0,  0,            0, 1      ]
    ])

Putting all angular matrices together we obtain:

In [30]:
def A(theta1, theta2, theta3, theta4, theta5):
    return A1(theta1) * A2(theta2) * A3(theta3) * A4(theta4) * A5(theta5)

Finally, for the transformation to hand from wrist, including x offset 57.75 and z offset 12.31, we have:
$$
T_w^h = \left[ \begin{array}{cccc}
                1 & 0 & 0 &  57.75 \\
                0 & 1 & 0 &   0    \\
                0 & 0 & 1 & -12.31 \\
                0 & 0 & 0 &   1
                \end{array} \right]
$$

In [31]:
T_h_w = matrix([[1, 0, 0,  0.05775], 
                [0, 1, 0,  0      ], 
                [0, 0, 1, -0.01231], 
                [0, 0, 0,  1      ]])

Hence we can calculate the forward kinematics using the following function:

In [112]:
def fk(theta1, theta2, theta3, theta4, theta5):
    #return T_s_o * A(theta1, theta2, theta3, theta4, theta5) * T_h_w * matrix([[0], [0], [0], [1]])
    return A(theta1, theta2, theta3, theta4, theta5) * T_h_w * matrix([[0], [0], [0], [1]])

In [242]:
fk(0.03225588798522949,
 0.015298128128051758,
 0.004559993743896484,
 0.03490658476948738,
 0.03072190284729004)

matrix([[ 0.0572133 ],
        [-0.10617721],
        [ 0.14619106],
        [ 1.        ]])

In [243]:
angles([[ 0.0572133,
         -0.10617721 - 0.098,
          0.14619106 + 0.10]], True)

{'RElbowRoll': 1.666666666666667, 'RElbowYaw': 1.1071487177940904, 'RShoulderRoll': 0.7110116899482348, 'RShoulderPitch': -1.2975892503764892}
Theta 1 (RShoulderPitch) valid: True
Theta 2 (RShoulderRoll) valid: False
Theta 3 (RElbowYaw) valid: True
Theta 4 (RElbowRoll) valid: False


In [229]:
angles([[0.05347554,
        -0.11188138 - 0.098,
         0.15109983 + 0.10]], True)

{'RElbowRoll': -1.0, 'RElbowYaw': 0, 'RShoulderRoll': 0.7117364018055012, 'RShoulderPitch': -1.3213151462473658}
Theta 1 (RShoulderPitch) valid: True
Theta 2 (RShoulderRoll) valid: False
Theta 3 (RElbowYaw) valid: True
Theta 4 (RElbowRoll) valid: False


In [1]:
tracker = ALProxy("ALTracker", IP, PORT)

NameError: name 'ALProxy' is not defined

In [22]:
proxy.getPosition("RHand", 0, True)

[0.19570279121398926,
 -0.10751388221979141,
 0.0018784627318382263,
 0.14232617616653442,
 0.40821823477745056,
 0.03787563368678093]

In [65]:
stiff("RArm")

In [21]:
proxy.setPositions("RArm", 0, [0.20292888581752777,
 -0.10282715409994125,
 0.01733347959816456,
 0.12099570780992508,
 0.32865583896636963,
 0.06073509901762009], MaxSpeed, 63)

In [64]:
unstiff("RArm")

In [33]:
fk(0.19570279121398926,
 -0.10751388221979141,
 0.0018784627318382263,
 0.14232617616653442,
 0.40821823477745056)

matrix([[ 0.04144972],
        [-0.07774094],
        [ 0.14645797],
        [ 1.        ]])

In [76]:
stiff("RArm")

In [77]:
time.sleep(0.1)
proxy.setPositions("RArm", 0, [0.5, -0.5, 0, 0, 0, 0], 0.2, 7)

In [78]:
unstiff("RArm")

In [79]:
tracker = ALProxy("ALTracker", IP, PORT)

In [80]:
stiff("RArm")

In [88]:
tracker.registerTarget("RedBall", 0.04)
tracker.track("RedBall")

In [81]:
unstiff("RArm")

In [104]:
targetPos = tracker.getTargetPosition(0)

In [105]:
stiff("RArm")

In [106]:
tracker.pointAt("RArm", targetPos, 0, 0.2)

In [111]:
unstiff("RArm")

In [102]:
targetPos

[0.18362721800804138, 0.03622455894947052, 0.04584433138370514]

In [107]:
sensor_angles = proxy.getAngles(["RArm"], True)

In [108]:
sensor_angles

[0.2316758632659912,
 0.28374814987182617,
 0.0014920234680175781,
 0.527738094329834,
 -0.1595778465270996,
 0.9571999907493591]

In [113]:
fk(0.2316758632659912,
 0.28374814987182617,
 0.0014920234680175781,
 0.527738094329834,
 -0.1595778465270996)

matrix([[ 0.03094863],
        [-0.01455497],
        [ 0.02233063],
        [ 1.        ]])

In [110]:
targetPos

[0.17745570838451385, 0.000478528905659914, 0.07570284605026245]